In [7]:
#!unzip drive/MyDrive/Pets-data/images.zip
#!cp -r drive/MyDrive/Pets-data/annotations .
#!ls

annotations  drive  images  sample_data


In [8]:
import os
from typing import Tuple

import torch
from PIL import Image
import numpy as np
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [9]:
class ImageDataset(Dataset):
  def __init__(self, filenames, labels):
    self.filenames = filenames
    self.labels = labels
    self.transformation = torchvision.models.ResNet34_Weights.IMAGENET1K_V1.transforms()


  def __len__(self):
    return len(self.filenames)


  def __getitem__(self, idx):
    image = Image.open(os.path.join('images', f'{self.filenames[idx]}.jpg')).convert('RGB')
    label = self.labels[idx]

    transformed_img = self.transformation(image)

    return transformed_img, label


In [10]:
def get_image_names_and_labels(filename):
    filenames, labels = [], []
    with open(filename) as f:
        lines = f.readlines()
        for line in lines:
            line_split = line.split()

            filenames.append(line_split[0])
            labels.append(int(line_split[2]) - 1)

    return filenames, labels



In [11]:
def get_pretrained_model_with_trainable_last_layer():
    model = torchvision.models.resnet34(weights=torchvision.models.ResNet34_Weights.IMAGENET1K_V1)

    for param in model.parameters():
        param.requires_grad = False

    model.fc = nn.Linear(in_features=model.fc.in_features, out_features=2)

    return model

In [12]:
model = get_pretrained_model_with_trainable_last_layer()

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:01<00:00, 81.1MB/s]


In [13]:
filenames, labels = get_image_names_and_labels('annotations/trainval.txt')
filenames_train, filenames_val, labels_train, labels_val = train_test_split(filenames, labels, test_size=0.2, stratify=labels)

dataset_train = ImageDataset(filenames_train, labels_train)
dataset_val = ImageDataset(filenames_val, labels_val)


train_data_loader = DataLoader(dataset_train, batch_size=32, shuffle=True)



In [16]:
num_epochs = 20
criterion = nn.CrossEntropyLoss()


for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for inputs, labels in tqdm(train_data_loader):
        #optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        #optimizer.step()
        train_loss += loss.item()

    print (train_loss)

 12%|█▏        | 11/92 [01:19<09:44,  7.22s/it]


KeyboardInterrupt: 